<a href="https://colab.research.google.com/github/Indresh0007/PySpark-Indresh/blob/main/Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## linux Basics

In [4]:
print("Hello")

Hello


In [6]:
!cat /etc/os-release


PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [7]:
!uname -a

Linux da354f09c9c4 6.1.123+ #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [8]:
!whoami

root


In [11]:
!pwd

/content


In [12]:
pwd


'/content'

## PySpark Basics of DataFrame

In [13]:
!pip install pyspark

In [14]:
!pip show pyspark

Name: pyspark
Version: 3.5.1
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: py4j
Required-by: dataproc-spark-connect


In [15]:
from pyspark.sql import SparkSession

In [16]:
spark =SparkSession.builder.appName("Basics").getOrCreate()

In [19]:
# Create DataFrame
data=[("Hello", "World")]
columns=["Word1","Word2"]

df = spark.createDataFrame(data, columns)

In [20]:
df.show()


+-----+-----+
|Word1|Word2|
+-----+-----+
|Hello|World|
+-----+-----+



## Basics Transformation and Action

In [23]:
columns = ["Name","Department","Salary"]
data = [
    ("John", "Sales", 3000),
    ("Jane", "Finance", 4000),
    ("Mike", "Sales", 3500),
    ("Alice", "Finance", 3800),
    ("Bob", "IT", 4500)
]

df=spark.createDataFrame(data, columns)

In [24]:
df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [25]:
# Filter: Employee with salary > 3500
df_filtered=df.filter(df["Salary"]>3500)
df_filtered.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| Jane|   Finance|  4000|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [27]:
df_grouped=df.groupBy("Department").avg("Salary")
df_grouped.show()

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3250.0|
|   Finance|     3900.0|
|        IT|     4500.0|
+----------+-----------+



In [28]:
#Add a new column: Salary with bonus (10%)
from pyspark.sql.functions import col
exp=col("Salary") * 1.1
df_with_bonus=df.withColumn("Salary_10%_bonus",exp)
df_with_bonus.show()

+-----+----------+------+------------------+
| Name|Department|Salary|  Salary_10%_bonus|
+-----+----------+------+------------------+
| John|     Sales|  3000|3300.0000000000005|
| Jane|   Finance|  4000|            4400.0|
| Mike|     Sales|  3500|3850.0000000000005|
|Alice|   Finance|  3800|            4180.0|
|  Bob|        IT|  4500|            4950.0|
+-----+----------+------+------------------+



In [34]:
from pyspark.sql.functions import col,upper,lower,concat_ws,when,length

df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [31]:
#Change Case transformation
df_upper = df.withColumn("Name_Upper", upper(col("Name")))
df_lower = df.withColumn("Name_Lower", lower(col("Name")))
df_upper.show()
df_lower.show()

+-----+----------+------+----------+
| Name|Department|Salary|Name_Upper|
+-----+----------+------+----------+
| John|     Sales|  3000|      JOHN|
| Jane|   Finance|  4000|      JANE|
| Mike|     Sales|  3500|      MIKE|
|Alice|   Finance|  3800|     ALICE|
|  Bob|        IT|  4500|       BOB|
+-----+----------+------+----------+

+-----+----------+------+----------+
| Name|Department|Salary|Name_Lower|
+-----+----------+------+----------+
| John|     Sales|  3000|      john|
| Jane|   Finance|  4000|      jane|
| Mike|     Sales|  3500|      mike|
|Alice|   Finance|  3800|     alice|
|  Bob|        IT|  4500|       bob|
+-----+----------+------+----------+



In [32]:
# Concatenate Columns
df_concat = df.withColumn("full_info", concat_ws("-", col("Name"), col("Department")))
df_concat.show()

+-----+----------+------+-------------+
| Name|Department|Salary|    full_info|
+-----+----------+------+-------------+
| John|     Sales|  3000|   John-Sales|
| Jane|   Finance|  4000| Jane-Finance|
| Mike|     Sales|  3500|   Mike-Sales|
|Alice|   Finance|  3800|Alice-Finance|
|  Bob|        IT|  4500|       Bob-IT|
+-----+----------+------+-------------+



In [35]:
# string length of names in new dataframe
df_length = df.withColumn("Name_Length", length(col("Name")))
df_length.show()

+-----+----------+------+-----------+
| Name|Department|Salary|Name_Length|
+-----+----------+------+-----------+
| John|     Sales|  3000|          4|
| Jane|   Finance|  4000|          4|
| Mike|     Sales|  3500|          4|
|Alice|   Finance|  3800|          5|
|  Bob|        IT|  4500|          3|
+-----+----------+------+-----------+



In [43]:
# conditional column (salary Category)
def_conditional = df.withColumn(
    "Salary_Category",
    when(col("Salary") >= 4000, "High")
    .when(col("Salary") >= 3500, "Medium")
    .otherwise("Low")

)
def_conditional.show()

+-----+----------+------+---------------+
| Name|Department|Salary|Salary_Category|
+-----+----------+------+---------------+
| John|     Sales|  3000|            Low|
| Jane|   Finance|  4000|           High|
| Mike|     Sales|  3500|         Medium|
|Alice|   Finance|  3800|         Medium|
|  Bob|        IT|  4500|           High|
+-----+----------+------+---------------+



In [46]:
# rename the column
def_renamed = df.withColumnRenamed("Salary", "Base_Salary")
def_renamed.show()

+-----+----------+-----------+
| Name|Department|Base_Salary|
+-----+----------+-----------+
| John|     Sales|       3000|
| Jane|   Finance|       4000|
| Mike|     Sales|       3500|
|Alice|   Finance|       3800|
|  Bob|        IT|       4500|
+-----+----------+-----------+

